In [11]:
import pandas as pd
import numpy as np
import math
import os

In [54]:
class Tarantula(object):

    def __init__(self, directory):

        self.__M = []  # boolean. coverage matrix
        self.__MT = []
        self.__F = []  # boolean. failing test cases
        self.__C = []
        self.__numOrigTests = 0
        self.__numStmts = 0
        self.__totalOrigFail = 0
        self.__totalOrigPass = 0
        self.__passOnStmt = []  # int. p(s), for every s and considering liveness of test cases
        self.__failOnStmt = []  # int. p(s), for every s and considering liveness of test cases
        self.__passRatio = []
        self.__failRatio = []
        self.__suspiciousness = []
        self.__directory = directory  # to store the coverage matrix file
        self.__ranking = []
        self.__fileName = []
        self.__rankData = []


    def main(self):
        
        # read file iterately under the root directory
        list = os.listdir(self.__directory) #list out all documents under the current directory
       #  print(len(list))
    
        for i in range(0,len(list)):
            self.__suspiciousness = []
            self.__fileName.insert(i, list[i])
            # print(list[i])
            path = os.path.join(self.__directory,list[i]) 
            if os.path.isfile(path):
                self.__M = pd.read_csv(path, sep=',', index_col=0)
                # initialize all case are live
                self.TarantulaSuspiciousnessCalculation(self.__M)

                # initialize the fail and pass case number
                self.setF(self.__M.r)

                # main computation
                self.mainCompute()
        i = 0
        print("==============The Results==============")
        for kv in self.__rankData:
            print(self.__fileName[i])
            print(kv)
            i += 1

            
    def TarantulaSuspiciousnessCalculation(self, M):
        self.__M = M

        self.__numOrigTests = self.__M.shape[0]
        # print(self.__numOrigTests)
        self.__numStmts = self.__M.shape[1] - 1
        # print(self.__numStmts)

    def setF(self, F):
        self.__F = F.tolist()
        self.calculateOrigFailAndPass()

    def mainCompute(self):
        self.calculatePassOnStmtAndFailOnStmt()
        self.calculatePassRatioAndFailRatio()
        self.calculateSuspiciousness()
        self.calculateRanking()
    
    def calculatePassOnStmtAndFailOnStmt(self):
        
        for i in range(self.__numStmts):
            self.__C = self.__M.iloc[:, i]
            self.__failOnStmt.insert(i, self.__C.tolist().count(0))
            self.__passOnStmt.insert(i, self.__C.tolist().count(1))

    def calculatePassRatioAndFailRatio(self):

        for i in range(self.__numStmts):
            if self.__totalOrigPass == 0:
                self.__passRatio.insert(i, 0)
            else:
                self.__passRatio.insert(i, self.__passOnStmt[i] / self.__totalOrigPass)

            if self.__totalOrigPass == 0:
                self.__failRatio.insert(i, 0)
            else:

                self.__failRatio.insert(i, self.__failOnStmt[i] / self.__totalOrigFail)

    def calculateSuspiciousness(self):
        
        for i in range(self.__numStmts):
            if (self.__totalOrigFail == 0) & (self.__totalOrigPass == 0):
                self.__suspiciousness.insert(i, -1)
            elif (self.__failRatio[i] == 0) & (self.__passRatio[i] == 0):
                self.__suspiciousness.insert(i, -1)
            else:
                self.__suspiciousness.insert(i, round(self.__failRatio[i] / (self.__failRatio[i] + self.__passRatio[i]), 2))

    def calculateOrigFailAndPass(self):
        
        for i in range(self.__numOrigTests):
            if self.__F[i]:

                self.__totalOrigPass += 1
            else:

                self.__totalOrigFail += 1

    def calculateRanking(self):
        # print whole list with suspiciousness and rank
        self.__MT = self.__M.T
        self.__suspiciousness.append(np.nan)
        self.__suspiciousness = pd.Series(self.__suspiciousness, index=[var for var in self.__M.columns])
        self.__ranking = self.__suspiciousness.rank(ascending=False, method="min")

        self.__MT["Suspiciousness"] = self.__suspiciousness
        self.__MT['Rank'] = self.__ranking
        self.__MT.sort_values("Rank", inplace=True)
        
        self.__rankData.append(self.__MT.head(5).index.tolist())

        # print(self.__MT)

In [55]:
if __name__ == "__main__":
    rootDirEncrypt = r"C:\Users\Chicago Lam\jupyter\encrypt_csv"
    rootDirSort = r"C:\Users\Chicago Lam\jupyter\sort_csv"
    encrypt = Tarantula(rootDirEncrypt)
    encrypt.main()
    sort = Tarantula(rootDirSort)
    sort.main()
    

==============The Results==============
encrypt_bug1(line13).csv
['s96', 's59', 's120', 's110', 's55']
encrypt_bug2(line17).csv
['s59', 's119', 's117', 's118', 's120']
encrypt_bug3(line27).csv
['s59', 's68', 's120', 's110', 's55']
encrypt_bug4(line37).csv
['s120', 's59', 's82', 's110', 's55']
encrypt_bug5(line43).csv
['s120', 's59', 's90', 's110', 's55']
==============The Results==============
bug_0_sort(line 6).csv
['s87', 's6', 's33', 's35', 's17']
bug_1_sort(line10).csv
['s6', 's33', 's35', 's17', 's41']
bug_2_sort(line17).csv
['s6', 's33', 's35', 's85', 's17']
bug_3_sort(line22).csv
['s6', 's33', 's35', 's17', 's41']
bug_4_sort(line25).csv
['s6', 's33', 's35', 's17', 's41']
